# Association Rules and Lift Analysis: D212
## By Natalie Toler

### Table of Contents
#### Part 1: Research Question
#### Part 2: Market Basket Justification
#### Part 3: Data Preparation and Analysis
#### Part 4: Data Summary and Implications
#### Part 5: Attachments

#### Sources

### Part 1: Research Question

A1. Question

   What combinations of medications are most frequently used together by patients, and how can the hospital system optimize its inventory and organization to ensure these combinations are readily available?

A2. Goal

   It is vital for hospitals to understand which medications are commonly used together to predict demand and ensure that the combinations are always in stock, improving patient care and efficency. The market basket analysis can identify frequent itemsets or associations between medications, providing data on which medications are often prescribed together. The hospital can use this information to better organize its med rooms, streamlining the process for healthcare providers and improving overall efficiency.

### Part 2: Market Basket Justification

B1. How it works and what the outcomes are

   Market Basket Analysis is a technique used to identify associations between items in a dataset. By transforming the dataset into a list of transactions, in this case, prescribed medications used by patients, the analysis can identify combinations of medications that appear together more frequently.

   This process uses three key metrics to determine significant combinations:

- Support: The frequency with which items appear together.

- Confidence: Measures how often the consequent (second item) appears in transactions containing the antecedent (first item). This metric helps to form association rules, which indicate relationships between medications, following the logic: "if X [the antecedent], then Y [the consequent]".

- Lift: Indicates the strength of the association compared to the expected frequency if the items were independent.

In this project, the expected outcomes of the analysis will be the identification of medication combinations, providing hospital staff with insights on how to optimize medication inventory. This ensures that frequently prescribed medications are in stock and organized in a way that streamlines the process for staff.

B2. One example of transactions in dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Import the data
df = pd.read_csv('medical_market_basket.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Presc01  7501 non-null   object
 1   Presc02  5747 non-null   object
 2   Presc03  4389 non-null   object
 3   Presc04  3345 non-null   object
 4   Presc05  2529 non-null   object
 5   Presc06  1864 non-null   object
 6   Presc07  1369 non-null   object
 7   Presc08  981 non-null    object
 8   Presc09  654 non-null    object
 9   Presc10  395 non-null    object
 10  Presc11  256 non-null    object
 11  Presc12  154 non-null    object
 12  Presc13  87 non-null     object
 13  Presc14  47 non-null     object
 14  Presc15  25 non-null     object
 15  Presc16  8 non-null      object
 16  Presc17  4 non-null      object
 17  Presc18  4 non-null      object
 18  Presc19  3 non-null      object
 19  Presc20  1 non-null      object
dtypes: object(20)
memory usage: 2.3+ MB


In [2]:
# Obtain an example of the transactions in the dataset
transaction_list = df.iloc[3].tolist()
print("\nFirst transaction (as list):")
print(transaction_list)


First transaction (as list):
['citalopram', 'benicar', 'amphetamine salt combo xr', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


We can see that the first transaction that has non-null data is transaction 3 in which the patient uses three medications: 

- [Citalopram](https://www.mayoclinic.org/drugs-supplements/citalopram-oral-route/description/drg-20062980), a common anti-depressant
- [Benicar](https://www.mayoclinic.org/drugs-supplements/olmesartan-oral-route/description/drg-20065169), a medication to treat high blood pressure
- [Amphetamine Salt Combo](https://www.mayoclinic.org/drugs-supplements/dextroamphetamine-and-amphetamine-oral-route/description/drg-20071758), a common treatment for ADHD

This is a good example of the importance of knowing medication pairings. Depression is a common comorbidity for patients with ADHD [Adult ADHD and comorbid disorders](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5567978/) and so in the market basket analysis if it is found that ADHD medications and Anti-depressants are often paired together this would be a reason to store adderall in the same section, or near, the anti-depressants. With these medications stored nearby hospital staff will have an easier time when treating a patient with these comorbidities.

B3. One assumption

   A Market Basket Analysis assumes that there is independence between transactions. One transaction should not influence another transation. This means that the anaylsis does not consider any relationship between the transactions, if a patient recieves a combinations of medications this does not directly affect the combination of medications that any other patient will recieve. This allows for the analysis to simply identify frequent itemsets and association rules based solely on the observed combinations within each individual transaction. This assumption is met within the dataset for this project as the transactions are based on the unique diagnonsis of each individual patient.

### Part 3: Data Preparation and Analysis

C1. Transform Dataset for Market Basket Analysis

In [3]:
#Inspect the DataFrame
df.head()

,Presc01,Presc02,Presc03,Presc04,Presc05,Presc06,Presc07,Presc08,Presc09,Presc10,Presc11,Presc12,Presc13,Presc14,Presc15,Presc16,Presc17,Presc18,Presc19,Presc20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,amlodipine,albuterol aerosol,allopurinol,pantoprazole,lorazepam,omeprazole,mometasone,fluconozole,gabapentin,pravastatin,cialis,losartan,metoprolol succinate XL,sulfamethoxazole,abilify,spironolactone,albuterol HFA,levofloxacin,promethazine,glipizide
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,citalopram,benicar,amphetamine salt combo xr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Drop all of the null rows
df = df[df['Presc01'].notna()]
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Presc01  7501 non-null   object
 1   Presc02  5747 non-null   object
 2   Presc03  4389 non-null   object
 3   Presc04  3345 non-null   object
 4   Presc05  2529 non-null   object
 5   Presc06  1864 non-null   object
 6   Presc07  1369 non-null   object
 7   Presc08  981 non-null    object
 8   Presc09  654 non-null    object
 9   Presc10  395 non-null    object
 10  Presc11  256 non-null    object
 11  Presc12  154 non-null    object
 12  Presc13  87 non-null     object
 13  Presc14  47 non-null     object
 14  Presc15  25 non-null     object
 15  Presc16  8 non-null      object
 16  Presc17  4 non-null      object
 17  Presc18  4 non-null      object
 18  Presc19  3 non-null      object
 19  Presc20  1 non-null      object
dtypes: object(20)
memory usage: 1.1+ MB


In [5]:
# Visually inspect the dataframe
df.head()

,Presc01,Presc02,Presc03,Presc04,Presc05,Presc06,Presc07,Presc08,Presc09,Presc10,Presc11,Presc12,Presc13,Presc14,Presc15,Presc16,Presc17,Presc18,Presc19,Presc20
0,amlodipine,albuterol aerosol,allopurinol,pantoprazole,lorazepam,omeprazole,mometasone,fluconozole,gabapentin,pravastatin,cialis,losartan,metoprolol succinate XL,sulfamethoxazole,abilify,spironolactone,albuterol HFA,levofloxacin,promethazine,glipizide
1,citalopram,benicar,amphetamine salt combo xr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,enalapril,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,paroxetine,allopurinol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abilify,atorvastatin,folic acid,naproxen,losartan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Create a list of lists and Check the output of the List of Lists
list_of_lists = df.apply(lambda row: [str(cell) for cell in row if not pd.isnull(cell)], axis=1).tolist()
list_of_lists

[['amlodipine',
  'albuterol aerosol',
  'allopurinol',
  'pantoprazole',
  'lorazepam',
  'omeprazole',
  'mometasone',
  'fluconozole',
  'gabapentin',
  'pravastatin',
  'cialis',
  'losartan',
  'metoprolol succinate XL',
  'sulfamethoxazole',
  'abilify',
  'spironolactone',
  'albuterol HFA',
  'levofloxacin',
  'promethazine',
  'glipizide'],
 ['citalopram', 'benicar', 'amphetamine salt combo xr'],
 ['enalapril'],
 ['paroxetine', 'allopurinol'],
 ['abilify', 'atorvastatin', 'folic acid', 'naproxen', 'losartan'],
 ['cialis'],
 ['hydrochlorothiazide', 'glyburide'],
 ['metformin', 'salmeterol inhaler', 'sertraline HCI'],
 ['metoprolol', 'carvedilol', 'losartan'],
 ['glyburide'],
 ['amphetamine salt combo xr', 'tramadol'],
 ['ezetimibe'],
 ['paroxetine',
  'citalopram',
  'abilify',
  'amphetamine salt combo xr',
  'fenofibrate'],
 ['carvedilol', 'Premarin', 'ezetimibe'],
 ['abilify', 'spironolactone'],
 ['abilify'],
 ['amlodipine',
  'diazepam',
  'clopidogrel',
  'metoprolol succi

In [7]:
# Create the encoder object and store the list of lists into a DataFrame
encoder = TransactionEncoder()
temp_array = encoder.fit(list_of_lists).transform(list_of_lists)
clean_df = pd.DataFrame(temp_array, columns = encoder.columns_)
pd.set_option('display.max_columns', None)
clean_df

,Duloxetine,Premarin,Yaz,abilify,acetaminophen,actonel,albuterol HFA,albuterol aerosol,alendronate,allopurinol,alprazolam,amitriptyline,amlodipine,amoxicillin,amphetamine,amphetamine salt combo,amphetamine salt combo xr,atenolol,atorvastatin,azithromycin,benazepril,benicar,boniva,bupropion sr,carisoprodol,carvedilol,cefdinir,celebrex,celecoxib,cephalexin,cialis,ciprofloxacin,citalopram,clavulanate K+,clonazepam,clonidine HCI,clopidogrel,clotrimazole,codeine,crestor,cyclobenzaprine,cymbalta,dextroamphetamine XR,diazepam,diclofenac sodium,doxycycline hyclate,enalapril,escitalopram,esomeprazole,ezetimibe,fenofibrate,fexofenadine,finasteride,flovent hfa 110mcg inhaler,fluconozole,fluoxetine HCI,fluticasone,fluticasone nasal spray,folic acid,furosemide,gabapentin,glimepiride,glipizide,glyburide,hydrochlorothiazide,hydrocodone,hydrocortisone 2.5% cream,ibuprophen,isosorbide mononitrate,lansoprazole,lantus,levofloxacin,levothyroxine sodium,lisinopril,lorazepam,losartan,lovastatin,meloxicam,metformin,metformin HCI,methylprednisone,metoprolol,metoprolol succinate XL,metoprolol tartrate,mometasone,naproxen,omeprazole,oxycodone,pantoprazole,paroxetine,pioglitazone,potassium Chloride,pravastatin,prednisone,pregabalin,promethazine,quetiapine,ranitidine,rosuvastatin,salmeterol inhaler,sertraline HCI,simvastatin,spironolactone,sulfamethoxazole,synthroid,tamsulosin,temezepam,topiramate,tramadol,trazodone HCI,triamcinolone Ace topical,triamterene,trimethoprim DS,valaciclovir,valsartan,venlafaxine XR,verapamil SR,viagra,zolpidem
0,False,False,False,True,False,False,True,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,True,False,True,False,True,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [8]:
# Check the cleaned dataset
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Columns: 119 entries, Duloxetine to zolpidem
dtypes: bool(119)
memory usage: 871.8 KB


In [9]:
# Write the cleaned dataframe to a csv
clean_df.to_csv('task3_clean_df.csv', index = False)

C2. Apriori Algorithm Code

In [10]:
# Using 0.03 as my support threshold isolate the frequent itemsets
frequent_itemset = apriori(clean_df, min_support = 0.03, use_colnames = True)
frequent_itemset

,support,itemsets
0,0.046794,(Premarin)
1,0.238368,(abilify)
2,0.033329,(allopurinol)
3,0.079323,(alprazolam)
4,0.071457,(amlodipine)
5,0.030129,(amphetamine)
6,0.068391,(amphetamine salt combo)
7,0.179709,(amphetamine salt combo xr)
8,0.129583,(atorvastatin)
9,0.174110,(carvedilol)


In [11]:
# Check how many rules have been preserved
print(len(clean_df.columns))
print(len(frequent_itemset))

119
54


C3. Values from Support, Lift, and Confidence of the Association Rules Table

In [12]:
# Create the Association Rules Table
rules = association_rules(frequent_itemset, metric = 'lift', min_threshold = 1.0)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(amphetamine salt combo xr),(abilify),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815,0.193648
1,(abilify),(amphetamine salt combo xr),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158,0.208562
2,(abilify),(atorvastatin),0.238368,0.129583,0.047994,0.201342,1.553774,0.017105,1.089850,0.467950
3,(atorvastatin),(abilify),0.129583,0.238368,0.047994,0.370370,1.553774,0.017105,1.209650,0.409465
4,(carvedilol),(abilify),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437
5,(abilify),(carvedilol),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
6,(abilify),(diazepam),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256,0.339197
7,(diazepam),(abilify),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357,0.308965
8,(abilify),(doxycycline hyclate),0.238368,0.095054,0.033729,0.141499,1.488616,0.011071,1.054100,0.430963
9,(doxycycline hyclate),(abilify),0.095054,0.238368,0.033729,0.354839,1.488616,0.011071,1.180529,0.362712


C4. Top Three Relevant Rules Generated by Apriori Code

In [13]:
rules.sort_values('lift', ascending = False).head(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
28,(carvedilol),(lisinopril),0.174110,0.098254,0.039195,0.225115,2.291162,0.022088,1.163716,0.682343
29,(lisinopril),(carvedilol),0.098254,0.174110,0.039195,0.398915,2.291162,0.022088,1.373997,0.624943
10,(abilify),(lisinopril),0.238368,0.098254,0.040928,0.171700,1.747522,0.017507,1.088672,0.561638


I am focusing on the lift metric to sort the top three rules. By focusing on the lift metric, I can highlight the strength of the associations, providing actionable insights for optimizing medication supply and organization within the hospital.

The top three rules decided by the lift metric are: 

1. Carvedilol -> Lisinopril
    The Antecedent is Carvedilol, the consequent is lisinopril. Therefore we could say that if Carvedilol, then Lisinopril. The Support metric tells us that 3.9% of the transactions contain this combination. In this case the confidence metric tells us that when carvedilol is prescribed there is a 22.51% chance that lisinopril is also prescribed. The lift metric means that prescribing carvedilol increases the likelihood of prescribing lisinopril by a factor of 2.29 compared to the liklihood of just prescribing lisinopril. Overall the rule indicates that patients prescribed carvedilol are significantly more likely to also be prescribed lisinopril. The lift value of 2.29 suggests a strong association between these two medications, meaning they are often used together.
    
2. Lisinopril -> Carvedilol
    This rule is the inverse of the first rule, indicating that the association between Lisinopril and Carvedilol is bidirectional. The confidence metric of this rule says that there is a 39.89% chance that carvedilol is prescribed when Lisinopril has been prescribed. This indicates that patients prescribed lisinopril are significantly more likely to also be prescribed carvedilol. The lift value of 2.29, the same as in Rule 1, reinforces the strong association between these two medications. This bidirectional relationship is important for ensuring that both medications are readily available when either one is prescribed.
    
3. Abilify -> Lisinopril
    The antecedent is Abilify, the consequent is lisinopril. Therefore we can say that if Abilify, then Lisinopril. The support metric shows that 4.09% of the transactions include both abilify and lisinopril. The confidence metric shows when abilify is prescribed, there is a 17.17% chance that lisinopril is also prescribed. Prescribing abilify increases the likelihood of prescribing lisinopril by a factor of 1.75 compared to the baseline likelihood of prescribing lisinopril. This rule indicates that patients prescribed abilify are more likely to also be prescribed lisinopril, though the association is not as strong as the first two rules. The lift value of 1.75 indicates a moderate association, suggesting that while the combination is not as common as the previous, it still occurs more frequently than would be expected by chance.
    
Carvedilol and Lisinopril are both blood pressure medicines. Abilify is an antipsychotic. [Drugs and Supplements](https://www.mayoclinic.org/drugs-supplements).

### Part 4: Data Summary and Implications

D1. Summarize the significance of support, lift, and confidence

- Support is the proportion of the transactions in the dataset that contain a particular itemset. This helps identify the frequency of a particular medication combination in the entire dataset. For example, the support of 3.92% for the combination of carvedilol and lisinopril means that 3.92% of all transactions included both medications. High support values suggest that the combination is relevant to a significant portion of the patient population.

- Confidence is the probability that a consequent is present in transactions that also contain the antecedent item. The reliability that the consequent medication is prescribed when the antecedent medication is prescribed. For example, the confidence of 39.89% for the rule "if lisinopril then carvedilol" means that in 39.89% of cases where lisinopril is prescribed, carvedilol is also prescribed. High confidence values imply a strong conditional relationship between the medications, suggesting that when one medication is prescribed, the other is likely to be prescribed as well.

- Lift is the ratio of the observed support to the expected support if the items were independent. It measures the strength of an association relative to random chance. Lift indicates how much more likely the consequent medication is to be prescribed when the antecedent medication is prescribed, compared to if the medications were independent. For example, the lift of 2.29 for the rule "if carvedilol then lisinopril" means that prescribing carvedilol increases the likelihood of prescribing lisinopril by 2.29 times compared to random chance. High lift values (greater than 1) indicate strong positive associations, suggesting that the medications are often used together.

D2. Practical Significance of Findings

   The three rules that were found provide some valuable insights for the hospital system. The strong association between carvedilol and lisinopril suggests that these medications should be kept in close proximity and adequately stocked together to meet patient needs. The moderate association between abilify and lisinopril suggests a less frequent, but still notable, co-prescription. Ensuring that these medications are easily accessible when one is prescribed can improve efficiency. These rules also give the hospital system a predictive pattern, understanding why these associations exist can help in predicting and anticipating the future medication needs of the patient population. 
   
D3. Course of Action

   The recommended course of action based on my question is that the hospital should ensure that they keep the blood pressure medications, carvedilol and lisinopril stocked and near each other in the med room. Keeping the blood pressure medications near the psychiatric medications would also be a good idea due to rule 3. However, there is one other actions I would recommend for the hospital system to even better explore the question of medication associations and med room stock. Since two of the three top rules include the same two medications which are both blood pressure medications it would be worth the time to use aggregation in an additional market basket analysis to explore how categories of medications are associated. Rule 3 suggests that psychiatric medications could be prescribed more often with blood pressure medications, or it could suggest that blood pressure medications are widely prescribed and abilify is a popular antipsychotic medication. Being able to look at associations between categories of medications would help to clarify the relationships.

### Part 5: Panopto Video

https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=4780d470-3bf1-4be8-a42b-b1a20056677a

### Sources

The DataCamp classes and D212 video lessons were used to create the code used in this project and the understanding of the different parts of the Market Basket Analysis. 

The Mayo Clinic Drugs and Supplements search were used to identify medications.

The National Library of Medicine study on Adult ADHD and comordbid disorders was used to confirm previous knowledge.

> Mayo Clinic. (n.d.). Drugs and supplements. Mayo Clinic. Retrieved July 2, 2024, from https://www.mayoclinic.org/drugs-supplements

> Katzman, M. A., Bilkey, T. S., Chokka, P. R., Fallu, A., & Klassen, L. J. (2017). Adult ADHD and Comorbid disorders: Clinical Implications of a Dimensional Approach. BMC Psychiatry, 17(1). https://doi.org/10.1186/s12888-017-1463-3

> Hull, I. (n.d.). Market Basket Analysis in Python. DataCamp; DataCamp. Retrieved July 2, 2024, from https://app.datacamp.com/learn/courses/market-basket-analysis-in-python

> Kamara, Dr. K. (n.d.). Market Basket Analysis in Theory. In Panopto. Retrieved July 2, 2024, from https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=9541a29b-2f14-4c5d-9d86-af030005bcf6

> Kamara, Dr. K. (n.d.). Market Basket Analysis in Python. In Panopto. Retrieved July 2, 2024, from https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=db85c4f1-0da5-4bde-a1a4-b07c0019d46d